In [23]:
import os
import json
import pandas as pd

In [26]:
import sqlite3

In [8]:
fc = json.loads(open("financings-contracts.json").read())
fd =  json.loads(open("financings-details.json").read())
fp =  json.loads(open("financings-payments.json").read())
fsi =  json.loads(open("financings-scheduled-instalments.json").read())
fw =  json.loads(open("financings-warranties.json").read())

In [9]:
elems = {
    'fc': fc,
    'fd': fd,
    'fp': fp,
    'fsi': fsi,
    'fw': fw,
}

In [10]:
for elem in elems:
    doc = elems[elem]
    size = len(set([i['customerId'] for i in doc]))
    print(elem, size, len(doc))

fc 500 500
fd 500 500
fp 500 500
fsi 500 500
fw 500 500


In [33]:
row = []
for d in fd:
    cid = d['customerId']
    doc = {
        'customerId': d['customerId'],
        'contractId': d['contractId'],
        'organizationId': d['organizationId'],
        'organizationName': d['organizationName'],
    }
    lfc = [i for i in fc if i['customerId'] == cid][0]
    lfd = [i for i in fd if i['customerId'] == cid][0]
    lfp = [i for i in fp if i['customerId'] == cid][0]
    lfsi = [i for i in fsi if i['customerId'] == cid][0]
    lfw = [i for i in fw if i['customerId'] == cid][0]
    
    doc['contractNumber'] = lfd['contractNumber']
    doc['productName'] = lfd['productName']
    doc['contractAmount'] = lfd['contractAmount']
    
    doc['fd_productType'] = lfd['productType']
    doc['fc_productType'] = lfc['productType']
    
    doc['warrantyType'] = lfw['warrantyType']
    doc['warrantySubType'] = lfw['warrantySubType']
    doc['warrantyAmount'] = lfw['warrantyAmount']
    
    doc['brandName'] = lfc['brandName']
    
    doc['totalNumberOfInstalments'] = lfsi['totalNumberOfInstalments']
    
    doc['fp_paidInstalments'] = lfp['paidInstalments']
    doc['fsi_paidInstalments'] = lfsi['paidInstalments']
    
    doc['contractOutstandingBalance'] = lfp['contractOutstandingBalance']
    
    fields = [
        #all
            #'customerId',
            #'contractId',
            #'organizationId',
            #'organizationName',
        # financings-details 
            'contractNumber',
            'productName',
            'contractAmount',
        'productType', # financings-details, financings-contracts
        # financings-warranties
            'warrantyType',
            'warrantySubType',
            'warrantyAmount',
        # financings-contracts
            'brandName',
        # financings-scheduled-instalments
            'totalNumberOfInstalments',
        # financings-payments, financings-scheduled-instalments
            'paidInstalments',
        # financings-payments
            'contractOutstandingBalance',
    ]
    
    row.append(doc)


In [34]:
len(row)

500

In [44]:
row[0]

{'customerId': '595.080.896-84',
 'contractId': 'c194d903-fca1-497f-b5f3-0204e426e798',
 'organizationId': '69665991-da55-4aac-a1f2-32d23daba8fe',
 'organizationName': 'Instituição Financeira 03',
 'contractNumber': 'c194d903-fca1-497f-b5f3-0204e426e798',
 'productName': 'Crédito Pessoal Consignado',
 'contractAmount': 736010.23,
 'fd_productType': 'FINANCIAMENTOS_IMOBILIARIOS',
 'fc_productType': 'FINANCIAMENTOS_IMOBILIARIOS',
 'warrantyType': 'CESSAO_DIREITOS_CREDITORIOS',
 'warrantySubType': 'ACOES_DEBENTURES',
 'warrantyAmount': 1472020.46,
 'brandName': 'Instituição Financeira 03',
 'totalNumberOfInstalments': 360,
 'fp_paidInstalments': 162,
 'fsi_paidInstalments': 162,
 'contractOutstandingBalance': 662409.207}

In [35]:
df = pd.DataFrame(row)

In [36]:
conn = sqlite3.connect('agro_insights')

In [37]:
c = conn.cursor()

In [39]:
c.execute('''
CREATE TABLE IF NOT EXISTS agro_insights (
    customerId string,
contractId string,
organizationId string,
organizationName string,
contractNumber string,
productName string,
contractAmount string,
fd_productType string,
fc_productType string,
warrantyType string,
warrantySubType string,
warrantyAmount string,
brandName string,
totalNumberOfInstalments string,
fp_paidInstalments string,
fsi_paidInstalments string,
contractOutstandingBalance string
)
''')
conn.commit()

In [40]:
df.to_sql('agro_insights', conn, if_exists='replace', index = False)

500

In [41]:
c.execute('''  
		SELECT * FROM agro_insights
          ''')

df22 = pd.DataFrame(c.fetchall()) 
print (df22)

                 0                                     1   \
0    595.080.896-84  c194d903-fca1-497f-b5f3-0204e426e798   
1    412.289.303-97  3528e870-9a85-4160-a81f-0aa20359f3c7   
2    785.797.738-73  e0432420-a9c9-450b-85ed-2753fbdfa4b0   
3    331.154.515-63  872c89e5-4234-4b0b-a0ae-30cd0f3e3e66   
4    114.795.138-29  ca78548d-b209-4134-9dd9-fa8eee2e5efd   
..              ...                                   ...   
495  504.337.567-56  a753bfd9-1a38-4649-b6b6-e262128063c1   
496  797.640.888-18  e8a300d2-025d-43dc-b0c3-bf137691aba5   
497  478.472.740-45  6362005a-ad64-4d57-91b8-97df9664df12   
498  651.841.821-25  1da402ce-d7a4-44ce-81ff-d625dc77b081   
499  219.778.534-52  6819adc4-df5a-4fa3-988b-e91bb3e6887d   

                                       2                          3   \
0    69665991-da55-4aac-a1f2-32d23daba8fe  Instituição Financeira 03   
1    69665991-da55-4aac-a1f2-32d23daba8fe  Instituição Financeira 03   
2    d3d2b0d5-903a-4f9f-b793-c139490d5ca7  Institui

In [31]:
for k in list(row[0].keys()):
    print(f"{k} string,")

customerId string,
contractId string,
organizationId string,
organizationName string,
contractNumber string,
productName string,
contractAmount string,
fd-productType string,
fc-productType string,
warrantyType string,
warrantySubType string,
warrantyAmount string,
brandName string,
totalNumberOfInstalments string,
fp-paidInstalments string,
fsi-paidInstalments string,
contractOutstandingBalance string,


In [25]:
conn = sqlite3.connect('agro_insights') 
c = conn.cursor()

c.execute('CREATE TABLE IF NOT EXISTS products (product_name text, price number)')
conn.commit()

data = {'product_name': ['Computer','Tablet','Monitor','Printer'],
        'price': [900,300,450,150]
        }

df = pd.DataFrame(data, columns= ['product_name','price'])
df.to_sql('products', conn, if_exists='replace', index = False)

,customerId,contractId,organizationId,organizationName,contractNumber,productName,contractAmount,fd-productType,fc-productType,warrantyType,warrantySubType,warrantyAmount,brandName,totalNumberOfInstalments,fp-paidInstalments,fsi-paidInstalments,contractOutstandingBalance
0,595.080.896-84,c194d903-fca1-497f-b5f3-0204e426e798,69665991-da55-4aac-a1f2-32d23daba8fe,Instituição Financeira 03,c194d903-fca1-497f-b5f3-0204e426e798,Crédito Pessoal Consignado,736010.23,FINANCIAMENTOS_IMOBILIARIOS,FINANCIAMENTOS_IMOBILIARIOS,CESSAO_DIREITOS_CREDITORIOS,ACOES_DEBENTURES,1472020.46,Instituição Financeira 03,360,162,162,662409.207
1,412.289.303-97,3528e870-9a85-4160-a81f-0aa20359f3c7,69665991-da55-4aac-a1f2-32d23daba8fe,Instituição Financeira 03,3528e870-9a85-4160-a81f-0aa20359f3c7,Crédito Pessoal Consignado,22434.20,FINANCIAMENTOS,FINANCIAMENTOS,GARANTIAS_INTERNACIONAIS,DIREITOS_SOBRE_ALUGUEIS,44868.40,Instituição Financeira 03,36,14,14,20190.780
2,785.797.738-73,e0432420-a9c9-450b-85ed-2753fbdfa4b0,d3d2b0d5-903a-4f9f-b793-c139490d5ca7,Instituição Financeira 09,e0432420-a9c9-450b-85ed-2753fbdfa4b0,Crédito Pessoal Consignado,1148287.62,FINANCIAMENTOS_IMOBILIARIOS,FINANCIAMENTOS_IMOBILIARIOS,SEGUROS_ASSEMELHADOS,CIVIL,2296575.24,Instituição Financeira 09,240,157,157,1033458.858
3,331.154.515-63,872c89e5-4234-4b0b-a0ae-30cd0f3e3e66,3fda1ef5-4b72-4557-bcbf-296780ba2c1f,Instituição Financeira 01,872c89e5-4234-4b0b-a0ae-30cd0f3e3e66,Crédito Pessoal Consignado,575257.04,FINANCIAMENTOS_RURAIS,FINANCIAMENTOS_RURAIS,GARANTIAS_INTERNACIONAIS,APLICACOES_FINANCEIRAS_RENDA_FIXA,1150514.08,Instituição Financeira 01,120,19,19,517731.336
4,114.795.138-29,ca78548d-b209-4134-9dd9-fa8eee2e5efd,e7fc1280-4dd3-4977-bc24-c08ffea8baf5,Instituição Financeira 06,ca78548d-b209-4134-9dd9-fa8eee2e5efd,Crédito Pessoal Consignado,1450390.02,FINANCIAMENTOS_IMOBILIARIOS,FINANCIAMENTOS_IMOBILIARIOS,OUTRAS_GARANTIAS_NAO_FIDEJUSSORIAS,CHEQUES,2900780.04,Instituição Financeira 06,360,110,110,1305351.018
